In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, average_precision_score, f1_score
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [2]:
from IPython.core.display import clear_output
import sys
def change_output(x):
    clear_output()
    sys.stdout.write(x)
    sys.stdout.flush()


In [3]:
def load_data(filename, sep=","):
    '''
        Функция для загрузки датасета без инициализации класса. 
        
        Возвращает:
         data - полный DataFrame
         fetures - список признаков для обучения/предсказания
         x - датасет для тренировки/предсказания в формате np.array
         y - целевой признак, если он есть. В случае с тестовым датасетом возвращает None
    '''
    
    data = pd.read_table(filename, sep=sep).dropna() #.fillna(-999999)    
    
    data = data[~(data['chainlen']>1000)]

    if ('DSSR' in data.columns):
        data.drop('DSSR', axis=1, inplace=True)    
    
    features = list(deepcopy(data.columns))
    [features.remove(column) for column in ['Id','index', 'pdb_chain', 'mg'] if column in data.columns];
    x_test = np.array(data[features])
    
    try:
        y_test = np.array(data['mg'])
    except: 
        y_test = None
    change_output('Data loaded')
    return {'data':data, 'features':features, 'x':x_test, 'y':y_test}

In [4]:
test = load_data("test.csv")

Data loaded

In [5]:
change_output('Loading data...')        
data = pd.read_table("train.csv")
change_output('Data processing...')  
if ('DSSR' in data.columns):
    data.drop('DSSR', axis=1, inplace=True)            
data = data.dropna()
y = deepcopy(np.array(np.matrix(data['mg']).flatten().tolist()[0])) 
data_numpy = np.matrix(data)
features = list(data.columns)
features.remove('pdb_chain')
features.remove('mg')
groups = np.array(data['pdb_chain'])
X = np.array(data[features])
features.append('mg')
change_output('Done')  

Done

In [6]:
X.shape

(209043, 383)

In [ ]:
test["x"].shape

In [ ]:
def xg_f1(y,t):
    t = t.get_label()
    y_bin = [1. if y_cont > 0.5 else 0. for y_cont in y] # binaryzing your output
    return 'f1',f1_score(t,y_bin)

In [ ]:
def loglikelood(preds, train_data):
    labels = train_data  #.get_label()
    preds = 1. / (1. + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1. - preds)
    return grad, hess

In [7]:
cat_feat = ((X[0] == 0.) | (X[0] == 1.)) & ((X[1] == 0.) | (X[1] == 1.))
cat_feat

array([ True, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [ ]:
clf.fit(X, y)
fi = clf.feature_importances_

In [8]:
fi = np.array([8.37719494e-04, 5.95851621e-03, 9.85790946e-03, 6.58311779e-04,
       4.98447384e-03, 5.34551361e-03, 4.41471291e-03, 4.50505970e-03,
       4.56330928e-03, 4.70680305e-03, 4.30524333e-03, 4.91976436e-03,
       3.97949899e-03, 4.37511247e-03, 4.07568193e-03, 5.17924926e-03,
       4.92749131e-03, 4.81285050e-03, 4.85455992e-03, 4.80210650e-03,
       5.44854764e-03, 4.88728381e-03, 4.26637682e-03, 4.53950962e-03,
       4.67869705e-03, 4.68349264e-03, 4.30915245e-03, 4.56332910e-03,
       4.13989792e-03, 4.64412344e-03, 5.55468678e-03, 4.52450085e-03,
       4.29287607e-03, 4.28285976e-03, 4.46584093e-03, 4.32664699e-03,
       4.51653001e-03, 4.21761114e-03, 4.27743202e-03, 4.64259729e-03,
       5.31728809e-03, 4.87377702e-03, 4.75601150e-03, 5.15440112e-03,
       5.53981827e-03, 5.23599622e-03, 5.26360711e-03, 3.91604110e-03,
       4.06798616e-03, 4.08266886e-03, 4.37467342e-03, 4.21931663e-03,
       4.46754553e-03, 4.24214126e-03, 4.55833046e-03, 5.33798445e-03,
       4.54425022e-03, 4.44415310e-03, 4.30843803e-03, 4.91772122e-03,
       4.17315336e-03, 4.95835987e-03, 3.80753214e-03, 4.39400751e-03,
       4.36322021e-03, 5.53643574e-03, 5.03440048e-03, 5.09679402e-03,
       5.20133254e-03, 5.62837785e-03, 5.36533506e-03, 5.14070197e-03,
       4.04494055e-03, 4.16946674e-03, 4.26667193e-03, 4.50308895e-03,
       3.96861546e-03, 4.59122884e-03, 4.20598830e-03, 4.69359791e-03,
       5.19369139e-03, 4.63584549e-03, 4.15966874e-03, 4.31123342e-03,
       4.58478757e-03, 4.35197617e-03, 4.79957404e-03, 4.19420419e-03,
       4.47094371e-03, 4.19985206e-03, 5.11594398e-03, 4.80625449e-03,
       4.79665905e-03, 4.88522230e-03, 5.61647547e-03, 5.47869701e-03,
       5.08340733e-03, 4.01876782e-03, 4.24213902e-03, 4.19247561e-03,
       4.34883928e-03, 4.39333467e-03, 4.37419072e-03, 4.04229371e-03,
       4.60332310e-03, 5.41950686e-03, 4.68108845e-03, 4.24900379e-03,
       4.34313570e-03, 4.47052198e-03, 4.55504200e-03, 4.67954354e-03,
       4.27798809e-03, 4.29317089e-03, 4.19199390e-03, 5.11875450e-03,
       4.90476795e-03, 4.86520413e-03, 5.01986410e-03, 5.43579538e-03,
       4.92259985e-03, 4.90694162e-03, 4.06197782e-03, 4.00283450e-03,
       4.25164962e-03, 4.42927435e-03, 4.16515525e-03, 4.46049020e-03,
       4.23435969e-03, 5.79104448e-04, 5.27844534e-04, 5.49401462e-04,
       5.08870959e-04, 5.43228006e-04, 5.26209835e-04, 6.33428325e-04,
       5.52223272e-04, 5.70007605e-04, 5.71919070e-04, 7.97155912e-04,
       4.84353490e-04, 6.61278601e-04, 7.31656713e-04, 9.63742407e-04,
       5.07483643e-04, 5.95337295e-04, 5.04932229e-04, 5.64716331e-04,
       4.53721215e-04, 5.69724214e-05, 9.99815893e-05, 1.83130567e-06,
       2.56696431e-05, 1.29364181e-04, 5.35290363e-05, 1.05337155e-04,
       2.07823015e-04, 3.66725195e-05, 1.16943202e-04, 3.79581850e-04,
       1.79645464e-05, 7.33507683e-07, 0.00000000e+00, 8.67001839e-06,
       1.71346338e-04, 0.00000000e+00, 9.68122952e-05, 6.73772053e-04,
       5.34915874e-04, 1.41833425e-04, 9.91809531e-05, 1.56198900e-04,
       4.41293854e-04, 1.09518792e-04, 6.45897969e-06, 3.48770882e-06,
       2.96577370e-04, 6.21098595e-05, 2.97335704e-04, 1.87282539e-04,
       5.87601197e-04, 2.10357337e-04, 1.26120027e-04, 3.66957905e-04,
       3.30279986e-04, 2.52522780e-04, 1.39314570e-04, 2.56529143e-04,
       1.49588667e-04, 4.00874365e-04, 6.13512153e-05, 1.62177644e-04,
       3.93548478e-06, 2.88900451e-05, 8.78022806e-05, 9.99540421e-05,
       1.48591171e-04, 1.80715757e-04, 6.26997490e-05, 1.79282217e-04,
       3.32733309e-04, 1.29912914e-05, 8.39516989e-06, 0.00000000e+00,
       1.05977155e-05, 1.93168155e-04, 0.00000000e+00, 7.91991931e-05,
       5.28009223e-04, 4.60317066e-04, 1.17261591e-04, 7.62299952e-05,
       2.98205775e-04, 4.09607879e-04, 4.94006310e-05, 2.41866080e-05,
       2.80139315e-06, 3.36784591e-04, 1.39955896e-04, 3.33939460e-04,
       2.18863761e-04, 5.88060757e-04, 2.90341727e-04, 2.14630452e-04,
       3.01876443e-04, 4.83256075e-04, 2.69988276e-04, 1.63472408e-04,
       2.59334517e-04, 1.63325507e-04, 3.54313832e-04, 4.89715332e-05,
       9.70264976e-05, 0.00000000e+00, 7.71810016e-06, 9.54130181e-05,
       9.89999999e-05, 7.92013869e-05, 1.60029549e-04, 1.04166914e-04,
       2.06765875e-04, 3.43121145e-04, 6.88432918e-06, 2.64602395e-05,
       0.00000000e+00, 4.83238088e-06, 1.41323649e-04, 0.00000000e+00,
       9.30730663e-05, 6.04684753e-04, 4.31345815e-04, 8.51820623e-05,
       6.26713544e-05, 1.18531357e-04, 4.02647854e-04, 4.46964015e-05,
       1.74753700e-05, 1.88540564e-05, 3.26131650e-04, 8.23416892e-05,
       3.43390024e-04, 1.91686590e-04, 6.04850045e-04, 2.75271081e-04,
       1.81471923e-04, 2.17965117e-04, 3.93605820e-04, 2.36572022e-04,
       1.70048983e-04, 3.43554932e-04, 1.49253958e-04, 3.58772052e-04,
       2.63894716e-05, 5.70927849e-05, 0.00000000e+00, 2.89809231e-05,
       6.47967885e-05, 4.11093221e-05, 2.14801414e-04, 2.40949632e-04,
       2.02792758e-05, 2.49642728e-04, 3.21244484e-04, 7.32023721e-06,
       5.66518255e-05, 0.00000000e+00, 2.88416959e-05, 1.56573049e-04,
       0.00000000e+00, 5.56516137e-05, 5.73642329e-04, 4.68682552e-04,
       7.86183739e-05, 4.97846765e-05, 1.23830568e-04, 3.54865972e-04,
       1.50719052e-04, 2.97068505e-05, 1.72614903e-04, 3.03014355e-04,
       2.43250969e-05, 2.25217687e-04, 1.91038080e-04, 5.52706982e-04,
       1.37675586e-04, 8.45958371e-05, 2.23918115e-04, 4.25967593e-04,
       2.27451305e-04, 1.79095533e-04, 3.42328660e-04, 1.17746551e-04,
       3.06990016e-04, 4.28498482e-05, 1.18261930e-04, 0.00000000e+00,
       4.37198954e-05, 6.08905326e-05, 7.51961905e-05, 1.12770761e-04,
       1.60020173e-04, 6.56277583e-05, 2.66325561e-04, 4.01114487e-04,
       2.99110348e-05, 1.85594664e-05, 0.00000000e+00, 1.54300968e-05,
       1.47432561e-04, 0.00000000e+00, 5.21852540e-05, 5.46555011e-04,
       5.10580254e-04, 6.57748306e-05, 1.27636661e-04, 1.01013032e-04,
       5.11901653e-04, 2.31260670e-04, 4.54754153e-06, 2.63174823e-04,
       2.52089795e-04, 6.17674758e-05, 1.81181880e-04, 1.87284511e-04,
       5.97785126e-04, 1.76717586e-04, 3.32998427e-05, 2.00395315e-04,
       4.75653510e-04, 2.43317803e-04, 1.81846200e-04, 3.72520671e-04,
       1.67174479e-04, 2.40488722e-04, 4.35844845e-04, 5.35013151e-04,
       2.40240392e-04, 5.08656837e-04, 5.44452082e-04, 3.09097311e-03,
       1.65267010e-03, 2.94408973e-02, 2.76366640e-02, 1.47814550e-02,
       2.10565143e-02, 2.10397296e-02, 2.31742452e-02, 1.08801911e-02,
       2.32715775e-02, 7.66467239e-03, 8.83440352e-03, 1.58065505e-02,
       1.42774157e-02, 7.64515305e-03, 1.21772694e-02, 2.84605206e-02,
       6.64886758e-03, 1.09290814e-02, 1.75631239e-02, 7.18848712e-03,
       2.01474919e-02, 2.10993687e-02, 0.00000000e+00])
fi

array([8.37719494e-04, 5.95851621e-03, 9.85790946e-03, 6.58311779e-04,
       4.98447384e-03, 5.34551361e-03, 4.41471291e-03, 4.50505970e-03,
       4.56330928e-03, 4.70680305e-03, 4.30524333e-03, 4.91976436e-03,
       3.97949899e-03, 4.37511247e-03, 4.07568193e-03, 5.17924926e-03,
       4.92749131e-03, 4.81285050e-03, 4.85455992e-03, 4.80210650e-03,
       5.44854764e-03, 4.88728381e-03, 4.26637682e-03, 4.53950962e-03,
       4.67869705e-03, 4.68349264e-03, 4.30915245e-03, 4.56332910e-03,
       4.13989792e-03, 4.64412344e-03, 5.55468678e-03, 4.52450085e-03,
       4.29287607e-03, 4.28285976e-03, 4.46584093e-03, 4.32664699e-03,
       4.51653001e-03, 4.21761114e-03, 4.27743202e-03, 4.64259729e-03,
       5.31728809e-03, 4.87377702e-03, 4.75601150e-03, 5.15440112e-03,
       5.53981827e-03, 5.23599622e-03, 5.26360711e-03, 3.91604110e-03,
       4.06798616e-03, 4.08266886e-03, 4.37467342e-03, 4.21931663e-03,
       4.46754553e-03, 4.24214126e-03, 4.55833046e-03, 5.33798445e-03,
      

In [9]:
fi_sorted = np.copy(fi)
fi_sorted.sort()

In [36]:
clf = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.01, scale_pos_weight=8.255102040816327)
cross_val_score(clf, X[:, fi > fi_sorted[10]], y, scoring="f1", cv=StratifiedKFold(shuffle = False, n_splits=5))

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

array([0.2915993 , 0.27296923, 0.25787339, 0.26928307, 0.32918726])

In [37]:
clf = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.01, scale_pos_weight=8.255102040816327)
cross_val_score(clf, X[:, fi > fi_sorted[20]], y, scoring="f1", cv=StratifiedKFold(shuffle = False, n_splits=5))

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

array([0.2915993 , 0.27296923, 0.25787339, 0.26928307, 0.32918726])

In [10]:
clf = lgb.LGBMClassifier(n_estimators=500, learning_rate=0.001, scale_pos_weight=8.255102040816327)
cross_val_score(clf, X[:, fi > fi_sorted[200]], y, scoring="f1", cv=StratifiedKFold(shuffle = False, n_splits=5))

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

array([0.27098858, 0.26966428, 0.25676667, 0.26558674, 0.28750793])

In [13]:
clf = CatBoostClassifier(scale_pos_weight=8.255102040816327, random_seed=18)
cross_val_score(clf, X, y, scoring="f1",
                fit_params={"cat_features":np.arange(len(cat_feat))[cat_feat]},
                cv=StratifiedKFold(shuffle = False, n_splits=5))
# array([0.28716937, 0.28160788, 0.26225178, 0.26626035, 0.32448428])

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/base.py:115: DeprecationWarning: Estimator CatBoostClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)


0:	learn: 0.6900025	total: 1.33s	remaining: 22m 12s
1:	learn: 0.6872478	total: 2.49s	remaining: 20m 42s
2:	learn: 0.6846699	total: 3.49s	remaining: 19m 19s
3:	learn: 0.6822575	total: 4.44s	remaining: 18m 26s
4:	learn: 0.6799998	total: 5.44s	remaining: 18m 2s
5:	learn: 0.6776489	total: 6.35s	remaining: 17m 32s
6:	learn: 0.6756599	total: 7.34s	remaining: 17m 21s
7:	learn: 0.6740132	total: 8.24s	remaining: 17m 1s
8:	learn: 0.6722299	total: 9.23s	remaining: 16m 55s
9:	learn: 0.6705655	total: 10.2s	remaining: 16m 52s
10:	learn: 0.6691246	total: 11.2s	remaining: 16m 48s
11:	learn: 0.6678184	total: 12.1s	remaining: 16m 38s
12:	learn: 0.6665904	total: 13s	remaining: 16m 28s
13:	learn: 0.6653037	total: 14s	remaining: 16m 26s
14:	learn: 0.6641643	total: 15s	remaining: 16m 24s
15:	learn: 0.6619751	total: 15.9s	remaining: 16m 16s
16:	learn: 0.6609046	total: 16.8s	remaining: 16m 9s
17:	learn: 0.6598613	total: 17.9s	remaining: 16m 16s
18:	learn: 0.6589494	total: 18.8s	remaining: 16m 10s
19:	learn: 0

154:	learn: 0.5981861	total: 2m 29s	remaining: 13m 37s
155:	learn: 0.5979997	total: 2m 30s	remaining: 13m 36s
156:	learn: 0.5971816	total: 2m 31s	remaining: 13m 34s
157:	learn: 0.5966544	total: 2m 33s	remaining: 13m 35s
158:	learn: 0.5964223	total: 2m 35s	remaining: 13m 42s
159:	learn: 0.5962570	total: 2m 37s	remaining: 13m 47s
160:	learn: 0.5960328	total: 2m 38s	remaining: 13m 46s
161:	learn: 0.5958425	total: 2m 39s	remaining: 13m 46s
162:	learn: 0.5956229	total: 2m 40s	remaining: 13m 46s
163:	learn: 0.5954368	total: 2m 42s	remaining: 13m 46s
164:	learn: 0.5951987	total: 2m 43s	remaining: 13m 46s
165:	learn: 0.5950391	total: 2m 44s	remaining: 13m 46s
166:	learn: 0.5945161	total: 2m 45s	remaining: 13m 46s
167:	learn: 0.5941012	total: 2m 46s	remaining: 13m 46s
168:	learn: 0.5939035	total: 2m 47s	remaining: 13m 45s
169:	learn: 0.5936785	total: 2m 49s	remaining: 13m 45s
170:	learn: 0.5933091	total: 2m 50s	remaining: 13m 45s
171:	learn: 0.5929709	total: 2m 51s	remaining: 13m 44s
172:	learn

304:	learn: 0.5636843	total: 5m 4s	remaining: 11m 33s
305:	learn: 0.5635280	total: 5m 5s	remaining: 11m 32s
306:	learn: 0.5634121	total: 5m 6s	remaining: 11m 31s
307:	learn: 0.5633035	total: 5m 7s	remaining: 11m 30s
308:	learn: 0.5631397	total: 5m 8s	remaining: 11m 29s
309:	learn: 0.5629385	total: 5m 9s	remaining: 11m 28s
310:	learn: 0.5627676	total: 5m 10s	remaining: 11m 26s
311:	learn: 0.5626169	total: 5m 10s	remaining: 11m 25s
312:	learn: 0.5624570	total: 5m 11s	remaining: 11m 24s
313:	learn: 0.5622792	total: 5m 12s	remaining: 11m 23s
314:	learn: 0.5621236	total: 5m 13s	remaining: 11m 22s
315:	learn: 0.5619844	total: 5m 14s	remaining: 11m 21s
316:	learn: 0.5618356	total: 5m 15s	remaining: 11m 20s
317:	learn: 0.5615938	total: 5m 16s	remaining: 11m 19s
318:	learn: 0.5613886	total: 5m 17s	remaining: 11m 17s
319:	learn: 0.5612188	total: 5m 18s	remaining: 11m 16s
320:	learn: 0.5610564	total: 5m 19s	remaining: 11m 15s
321:	learn: 0.5608547	total: 5m 20s	remaining: 11m 14s
322:	learn: 0.56

456:	learn: 0.5400538	total: 7m 28s	remaining: 8m 53s
457:	learn: 0.5399169	total: 7m 29s	remaining: 8m 52s
458:	learn: 0.5397960	total: 7m 30s	remaining: 8m 51s
459:	learn: 0.5396787	total: 7m 31s	remaining: 8m 50s
460:	learn: 0.5394283	total: 7m 32s	remaining: 8m 49s
461:	learn: 0.5392872	total: 7m 33s	remaining: 8m 48s
462:	learn: 0.5391848	total: 7m 34s	remaining: 8m 47s
463:	learn: 0.5390454	total: 7m 35s	remaining: 8m 46s
464:	learn: 0.5388744	total: 7m 36s	remaining: 8m 45s
465:	learn: 0.5387382	total: 7m 37s	remaining: 8m 44s
466:	learn: 0.5385669	total: 7m 38s	remaining: 8m 43s
467:	learn: 0.5383503	total: 7m 39s	remaining: 8m 41s
468:	learn: 0.5382644	total: 7m 40s	remaining: 8m 41s
469:	learn: 0.5381416	total: 7m 41s	remaining: 8m 40s
470:	learn: 0.5378890	total: 7m 42s	remaining: 8m 38s
471:	learn: 0.5377573	total: 7m 42s	remaining: 8m 37s
472:	learn: 0.5376499	total: 7m 43s	remaining: 8m 36s
473:	learn: 0.5375289	total: 7m 44s	remaining: 8m 35s
474:	learn: 0.5373990	total:

...

456:	learn: 0.5525851	total: 7m 41s	remaining: 9m 8s
457:	learn: 0.5525025	total: 7m 42s	remaining: 9m 7s
458:	learn: 0.5524249	total: 7m 44s	remaining: 9m 7s
459:	learn: 0.5522020	total: 7m 45s	remaining: 9m 6s
460:	learn: 0.5521074	total: 7m 46s	remaining: 9m 5s
461:	learn: 0.5520276	total: 7m 47s	remaining: 9m 4s
462:	learn: 0.5518960	total: 7m 48s	remaining: 9m 3s
463:	learn: 0.5518240	total: 7m 49s	remaining: 9m 2s
464:	learn: 0.5517781	total: 7m 50s	remaining: 9m 1s
465:	learn: 0.5517127	total: 7m 51s	remaining: 9m
466:	learn: 0.5516282	total: 7m 52s	remaining: 8m 59s
467:	learn: 0.5515706	total: 7m 54s	remaining: 8m 58s
468:	learn: 0.5514782	total: 7m 55s	remaining: 8m 58s
469:	learn: 0.5513349	total: 7m 56s	remaining: 8m 57s
470:	learn: 0.5512517	total: 7m 57s	remaining: 8m 56s
471:	learn: 0.5511162	total: 7m 58s	remaining: 8m 55s
472:	learn: 0.5510072	total: 7m 59s	remaining: 8m 54s
473:	learn: 0.5509457	total: 8m	remaining: 8m 53s
474:	learn: 0.5509105	total: 8m 1s	remaining:

609:	learn: 0.5423203	total: 10m 33s	remaining: 6m 44s
610:	learn: 0.5422915	total: 10m 34s	remaining: 6m 43s
611:	learn: 0.5422680	total: 10m 35s	remaining: 6m 42s
612:	learn: 0.5422406	total: 10m 36s	remaining: 6m 41s
613:	learn: 0.5421895	total: 10m 37s	remaining: 6m 40s
614:	learn: 0.5421468	total: 10m 38s	remaining: 6m 39s
615:	learn: 0.5420970	total: 10m 39s	remaining: 6m 38s
616:	learn: 0.5420696	total: 10m 40s	remaining: 6m 37s
617:	learn: 0.5420464	total: 10m 41s	remaining: 6m 36s
618:	learn: 0.5420085	total: 10m 43s	remaining: 6m 35s
619:	learn: 0.5419798	total: 10m 44s	remaining: 6m 34s
620:	learn: 0.5419596	total: 10m 45s	remaining: 6m 33s
621:	learn: 0.5418459	total: 10m 46s	remaining: 6m 32s
622:	learn: 0.5417471	total: 10m 47s	remaining: 6m 31s
623:	learn: 0.5417386	total: 10m 48s	remaining: 6m 30s
624:	learn: 0.5416541	total: 10m 49s	remaining: 6m 29s
625:	learn: 0.5416319	total: 10m 50s	remaining: 6m 28s
626:	learn: 0.5416051	total: 10m 52s	remaining: 6m 27s
627:	learn

760:	learn: 0.5375400	total: 13m 21s	remaining: 4m 11s
761:	learn: 0.5375327	total: 13m 22s	remaining: 4m 10s
762:	learn: 0.5375178	total: 13m 23s	remaining: 4m 9s
763:	learn: 0.5374924	total: 13m 24s	remaining: 4m 8s
764:	learn: 0.5374811	total: 13m 25s	remaining: 4m 7s
765:	learn: 0.5374715	total: 13m 26s	remaining: 4m 6s
766:	learn: 0.5374219	total: 13m 27s	remaining: 4m 5s
767:	learn: 0.5374047	total: 13m 29s	remaining: 4m 4s
768:	learn: 0.5373877	total: 13m 30s	remaining: 4m 3s
769:	learn: 0.5373698	total: 13m 31s	remaining: 4m 2s
770:	learn: 0.5373553	total: 13m 32s	remaining: 4m 1s
771:	learn: 0.5373393	total: 13m 33s	remaining: 4m
772:	learn: 0.5373320	total: 13m 34s	remaining: 3m 59s
773:	learn: 0.5373161	total: 13m 35s	remaining: 3m 58s
774:	learn: 0.5373085	total: 13m 36s	remaining: 3m 57s
775:	learn: 0.5372961	total: 13m 37s	remaining: 3m 56s
776:	learn: 0.5372782	total: 13m 38s	remaining: 3m 55s
777:	learn: 0.5372504	total: 13m 40s	remaining: 3m 54s
778:	learn: 0.5372347	t

911:	learn: 0.5353124	total: 16m 9s	remaining: 1m 33s
912:	learn: 0.5352952	total: 16m 10s	remaining: 1m 32s
913:	learn: 0.5352832	total: 16m 11s	remaining: 1m 31s
914:	learn: 0.5352750	total: 16m 12s	remaining: 1m 30s
915:	learn: 0.5352673	total: 16m 13s	remaining: 1m 29s
916:	learn: 0.5352653	total: 16m 14s	remaining: 1m 28s
917:	learn: 0.5352497	total: 16m 16s	remaining: 1m 27s
918:	learn: 0.5352368	total: 16m 17s	remaining: 1m 26s
919:	learn: 0.5352095	total: 16m 18s	remaining: 1m 25s
920:	learn: 0.5351959	total: 16m 19s	remaining: 1m 24s
921:	learn: 0.5351784	total: 16m 20s	remaining: 1m 22s
922:	learn: 0.5351432	total: 16m 21s	remaining: 1m 21s
923:	learn: 0.5351277	total: 16m 22s	remaining: 1m 20s
924:	learn: 0.5351092	total: 16m 23s	remaining: 1m 19s
925:	learn: 0.5350994	total: 16m 24s	remaining: 1m 18s
926:	learn: 0.5350846	total: 16m 25s	remaining: 1m 17s
927:	learn: 0.5350742	total: 16m 26s	remaining: 1m 16s
928:	learn: 0.5350573	total: 16m 28s	remaining: 1m 15s
929:	learn:

array([0.28716937, 0.28160788, 0.26225178, 0.26626035, 0.32448428])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)
clf = 17

In [7]:
del clf
clf = xgb.XGBClassifier(scale_pos_weight=8.255102040816327, n_estimators=600, gamma=10, random_state=17)
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=10, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=17, reg_alpha=0, reg_lambda=1,
       scale_pos_weight=8.255102040816327, seed=None, silent=True,
       subsample=1)

In [8]:
print(f1_score(clf.predict(X_test), y_test))
# 0.3310046910330397 (scale_pos_weight=8.255102040816327, n_estimators=100, gamma=0)
# 0.333  (scale_pos_weight=8.255102040816327, n_estimators=100, gamma=1)
# 0.32618517769235955 (scale_pos_weight=8.255102040816327, n_estimators=100, gamma=5)
# 0.3354528850706893 (scale_pos_weight=8.255102040816327, n_estimators=100, gamma=10)
# 0.3817950956273312 (scale_pos_weight=8.255102040816327, n_estimators=300, gamma=10)
# 0.42455511346246333 (scale_pos_weight=8.255102040816327, n_estimators=600, gamma=10, random_state=17)

0.42455511346246333


/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
clf = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.01, scale_pos_weight=2.0)
params = {
    "n_estimators":[300],
    "learning_rate":[0.01], #0.0005, 0.001, 0.005, 0.01, 0.05],
    "scale_pos_weight":[8.255102040816327],
    "boosting_type":['gbdt'],
}

gscv = GridSearchCV(clf, params, scoring="f1", cv=StratifiedKFold(shuffle = False, n_splits=5))
gscv.fit(X, y)

In [42]:
%%time
np.arange(1, 10)

CPU times: user 39 µs, sys: 0 ns, total: 39 µs
Wall time: 47.2 µs


array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [28]:
import os
os.system('spd-say "Acknowledged"')

-1

In [19]:
gscv.grid_scores_

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.28876, std: 0.02667, params: {'boosting_type': 'gbdt', 'learning_rate': 0.01, 'n_estimators': 750, 'scale_pos_weight': 9.0},
 mean: 0.27868, std: 0.01830, params: {'boosting_type': 'dart', 'learning_rate': 0.01, 'n_estimators': 750, 'scale_pos_weight': 9.0},
 mean: 0.29242, std: 0.02722, params: {'boosting_type': 'goss', 'learning_rate': 0.01, 'n_estimators': 750, 'scale_pos_weight': 9.0}]

In [20]:
gscv.best_score_

0.29242072490189625

In [21]:
gscv.best_params_

{'boosting_type': 'goss',
 'learning_rate': 0.01,
 'n_estimators': 750,
 'scale_pos_weight': 9.0}

In [ ]:
# array([0.00772472, 0.00244328, 0.00176118, 0.238831  , 0.00281096]) RF 100
# array([0.1082544 , 0.11260222, 0.05920281, 0.26041424, 0.09496811]) DT 15
# array([0.0664723 , 0.07678245, 0.0174002 , 0.27320219, 0.03447715]) DT 10
# array([0.03578187, 0.01340897, 0.00581879, 0.00132802, 0.00586142]) CatBoost
# array([0.08434712, 0.05030018, 0.01561858, 0.00582165, 0.02612778]) CatBoost 3k
# array([0.10094637, 0.06313806, 0.02475649, 0.01292024, 0.03391199]) CatBoost 6k -> 0.18848
# ... CatBoost 9k -> 0.20512
# array([0.19595154, 0.19486361, 0.13402873, 0.24864054, 0.17646506]) DT -> 0.20076
# (array([0.15511116, 0.1459787 , 0.09896751, 0.25948915, 0.14122632]), 0.16015456743463227) DT md = 20
# (array([0.09605055, 0.11355198, 0.05333333, 0.07721804, 0.09993754]), 0.0880182881326547) ETC 
# (array([0.08023579, 0.10090881, 0.04891122, 0.06303872, 0.09434263]), 0.07748743541127794) ETC ne = 20

# (array([0.0952381 , 0.05911174, 0.01544269, 0.23348956, 0.04101367]), 0.08885915227716767) lgb ne 1000
# (array([0.08036001, 0.05293441, 0.0295868 , 0.23099507, 0.05625414]), 0.09002608666932135) lgb ne 10000 -> 0.19689
"""
[mean: 0.05323, std: 0.10577, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 0.1},
 mean: 0.06368, std: 0.09360, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 2.138775510204082},
 mean: 0.21756, std: 0.04924, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 4.177551020408163},
 mean: 0.28336, std: 0.03686, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 6.216326530612245},
 mean: 0.28418, std: 0.02498, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 8.255102040816327},
 mean: 0.27476, std: 0.01649, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 10.29387755102041},
 mean: 0.26686, std: 0.01281, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 12.332653061224491},
 mean: 0.26051, std: 0.00943, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 14.371428571428572},
 mean: 0.25459, std: 0.00911, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 16.410204081632656},
 mean: 0.25052, std: 0.00854, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 18.44897959183674},
 mean: 0.24771, std: 0.00776, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 20.487755102040822},
 mean: 0.24520, std: 0.00752, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 22.5265306122449},
 mean: 0.24357, std: 0.00746, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 24.565306122448984},
 mean: 0.24230, std: 0.00696, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 26.604081632653067},
 mean: 0.24141, std: 0.00649, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 28.642857142857146},
 mean: 0.24099, std: 0.00645, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 30.68163265306123},
 mean: 0.24033, std: 0.00659, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 32.72040816326531},
 mean: 0.24001, std: 0.00650, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 34.759183673469394},
 mean: 0.23971, std: 0.00622, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 36.79795918367348},
 mean: 0.23942, std: 0.00630, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 38.83673469387756},
 mean: 0.23914, std: 0.00604, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 40.87551020408164},
 mean: 0.23893, std: 0.00613, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 42.91428571428572},
 mean: 0.23895, std: 0.00575, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 44.9530612244898},
 mean: 0.23884, std: 0.00562, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 46.991836734693884},
 mean: 0.23861, std: 0.00563, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 49.030612244897966},
 mean: 0.23875, std: 0.00551, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 51.06938775510205},
 mean: 0.23881, std: 0.00554, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 53.10816326530613},
 mean: 0.23869, std: 0.00550, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 55.146938775510215},
 mean: 0.23872, std: 0.00552, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 57.18571428571429},
 mean: 0.23864, std: 0.00527, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 59.22448979591837},
 mean: 0.23866, std: 0.00518, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 61.263265306122456},
 mean: 0.23858, std: 0.00518, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 63.30204081632654},
 mean: 0.23858, std: 0.00511, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 65.34081632653061},
 mean: 0.23855, std: 0.00496, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 67.37959183673469},
 mean: 0.23852, std: 0.00509, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 69.41836734693878},
 mean: 0.23855, std: 0.00503, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 71.45714285714286},
 mean: 0.23844, std: 0.00495, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 73.49591836734695},
 mean: 0.23836, std: 0.00500, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 75.53469387755102},
 mean: 0.23841, std: 0.00496, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 77.57346938775511},
 mean: 0.23845, std: 0.00485, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 79.61224489795919},
 mean: 0.23834, std: 0.00495, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 81.65102040816328},
 mean: 0.23836, std: 0.00481, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 83.68979591836735},
 mean: 0.23830, std: 0.00479, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 85.72857142857143},
 mean: 0.23834, std: 0.00483, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 87.76734693877552},
 mean: 0.23836, std: 0.00479, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 89.8061224489796},
 mean: 0.23844, std: 0.00481, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 91.84489795918368},
 mean: 0.23827, std: 0.00483, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 93.88367346938776},
 mean: 0.23833, std: 0.00469, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 95.92244897959185},
 mean: 0.23839, std: 0.00475, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 97.96122448979592},
 mean: 0.23829, std: 0.00484, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 100.0}]
 
 
 gscv.best_score_

0.2841823044383953  -> 0.34377

gscv.best_params_

{'learning_rate': 0.01,
 'n_estimators': 300,
 'scale_pos_weight': 8.255102040816327}
 """

"""
[mean: 0.20706, std: 0.05033, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 4.0},
 mean: 0.26685, std: 0.04386, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 5.333333333333333},
 mean: 0.28560, std: 0.03334, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 6.666666666666666},
 mean: 0.28459, std: 0.02570, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 8.0},
 mean: 0.27991, std: 0.01930, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 9.333333333333332},
 mean: 0.27388, std: 0.01632, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 10.666666666666666},
 mean: 0.26811, std: 0.01353, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 12.0},
 mean: 0.26414, std: 0.01186, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 13.333333333333332},
 mean: 0.25941, std: 0.00942, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 14.666666666666666},
 mean: 0.25501, std: 0.00953, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 16.0},
 mean: 0.22041, std: 0.04872, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 4.0},
 mean: 0.26731, std: 0.04250, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 5.333333333333333},
 mean: 0.28627, std: 0.03440, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 6.666666666666666},
 mean: 0.28922, std: 0.02897, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 8.0},
 mean: 0.28460, std: 0.02475, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 9.333333333333332},
 mean: 0.28032, std: 0.02167, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 10.666666666666666},
 mean: 0.27551, std: 0.01801, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 12.0},
 mean: 0.27217, std: 0.01669, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 13.333333333333332},
 mean: 0.26754, std: 0.01392, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 14.666666666666666},
 mean: 0.26370, std: 0.01351, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 16.0},
 mean: 0.22822, std: 0.04405, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 4.0},
 mean: 0.26802, std: 0.03971, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.28289, std: 0.03304, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.28963, std: 0.02979, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 8.0},
 mean: 0.28981, std: 0.02785, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.28772, std: 0.02674, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.28451, std: 0.02468, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 12.0},
 mean: 0.28119, std: 0.02152, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27769, std: 0.01985, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.27465, std: 0.01897, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 16.0},
 mean: 0.20395, std: 0.04719, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 4.0},
 mean: 0.23063, std: 0.03735, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.25114, std: 0.03337, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.26193, std: 0.02488, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 8.0},
 mean: 0.27229, std: 0.02798, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.27565, std: 0.02612, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.27785, std: 0.02704, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 12.0},
 mean: 0.27886, std: 0.02668, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27799, std: 0.02680, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.28051, std: 0.02716, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 16.0},
 mean: 0.23492, std: 0.04060, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 4.0},
 mean: 0.26552, std: 0.03649, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 5.333333333333333},
 mean: 0.28126, std: 0.03310, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 6.666666666666666},
 mean: 0.28760, std: 0.03033, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 8.0},
 mean: 0.28598, std: 0.03007, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 9.333333333333332},
 mean: 0.28809, std: 0.02763, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 10.666666666666666},
 mean: 0.28643, std: 0.02514, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 12.0},
 mean: 0.28320, std: 0.02437, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27911, std: 0.02237, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 14.666666666666666},
 mean: 0.27837, std: 0.02229, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 16.0},
 mean: 0.22883, std: 0.04080, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 4.0},
 mean: 0.25795, std: 0.03532, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 5.333333333333333},
 mean: 0.27354, std: 0.03411, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 6.666666666666666},
 mean: 0.28370, std: 0.03061, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 8.0},
 mean: 0.28281, std: 0.03125, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 9.333333333333332},
 mean: 0.28486, std: 0.02903, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 10.666666666666666},
 mean: 0.28626, std: 0.02596, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 12.0},
 mean: 0.28394, std: 0.02681, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 13.333333333333332},
 mean: 0.28263, std: 0.02496, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 14.666666666666666},
 mean: 0.28228, std: 0.02642, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 16.0},
 mean: 0.20808, std: 0.04105, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 4.0},
 mean: 0.23557, std: 0.03610, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.25194, std: 0.03524, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.26022, std: 0.03107, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 8.0},
 mean: 0.26884, std: 0.03023, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.27357, std: 0.02886, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.27729, std: 0.02388, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 12.0},
 mean: 0.27789, std: 0.02748, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27606, std: 0.02205, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.27736, std: 0.02499, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 16.0},
 mean: 0.14304, std: 0.05671, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 4.0},
 mean: 0.15952, std: 0.05062, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.17057, std: 0.04808, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.18220, std: 0.04403, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 8.0},
 mean: 0.18981, std: 0.04020, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.19500, std: 0.03806, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.20141, std: 0.03955, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 12.0},
 mean: 0.20449, std: 0.03588, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.20733, std: 0.03551, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.21532, std: 0.03224, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 16.0},
 mean: 0.22094, std: 0.04181, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 4.0},
 mean: 0.25424, std: 0.03253, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 5.333333333333333},
 mean: 0.26768, std: 0.03290, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 6.666666666666666},
 mean: 0.27622, std: 0.03376, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 8.0},
 mean: 0.28154, std: 0.02565, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 9.333333333333332},
 mean: 0.28383, std: 0.02564, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 10.666666666666666},
 mean: 0.28273, std: 0.02643, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 12.0},
 mean: 0.28080, std: 0.02688, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27759, std: 0.02513, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 14.666666666666666},
 mean: 0.28004, std: 0.02120, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 16.0},
 mean: 0.20670, std: 0.04570, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 4.0},
 mean: 0.23650, std: 0.03530, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 5.333333333333333},
 mean: 0.25050, std: 0.03375, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 6.666666666666666},
 mean: 0.26208, std: 0.03096, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 8.0},
 mean: 0.26656, std: 0.02353, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 9.333333333333332},
 mean: 0.27131, std: 0.02730, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 10.666666666666666},
 mean: 0.27289, std: 0.02757, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 12.0},
 mean: 0.27570, std: 0.02684, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27408, std: 0.02351, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 14.666666666666666},
 mean: 0.27256, std: 0.02482, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 16.0},
 mean: 0.18596, std: 0.04930, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 4.0},
 mean: 0.20549, std: 0.03619, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.22255, std: 0.03728, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.23551, std: 0.03574, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 8.0},
 mean: 0.24121, std: 0.03051, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.24589, std: 0.02480, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.24879, std: 0.02580, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 12.0},
 mean: 0.25603, std: 0.03071, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.25319, std: 0.02462, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.25446, std: 0.02384, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 16.0},
 mean: 0.12828, std: 0.06079, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 4.0},
 mean: 0.13462, std: 0.05532, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.14255, std: 0.05321, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.15224, std: 0.05075, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 8.0},
 mean: 0.15668, std: 0.05082, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.15897, std: 0.04729, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.16672, std: 0.04653, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 12.0},
 mean: 0.17135, std: 0.04642, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.17110, std: 0.04322, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.16905, std: 0.04374, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 16.0},
 mean: 0.19312, std: 0.04069, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 4.0},
 mean: 0.21341, std: 0.03160, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 5.333333333333333},
 mean: 0.22072, std: 0.03369, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 6.666666666666666},
 mean: 0.21744, std: 0.03032, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 8.0},
 mean: 0.22477, std: 0.02650, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 9.333333333333332},
 mean: 0.22657, std: 0.02755, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 10.666666666666666},
 mean: 0.23377, std: 0.03099, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 12.0},
 mean: 0.23871, std: 0.02639, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 13.333333333333332},
 mean: 0.22775, std: 0.02453, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 14.666666666666666},
 mean: 0.23320, std: 0.02443, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 16.0},
 mean: 0.18128, std: 0.04470, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 4.0},
 mean: 0.19416, std: 0.03488, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 5.333333333333333},
 mean: 0.20412, std: 0.03998, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 6.666666666666666},
 mean: 0.20243, std: 0.03347, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 8.0},
 mean: 0.20433, std: 0.03390, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 9.333333333333332},
 mean: 0.20763, std: 0.03218, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 10.666666666666666},
 mean: 0.21488, std: 0.03639, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 12.0},
 mean: 0.22009, std: 0.03097, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 13.333333333333332},
 mean: 0.21002, std: 0.03417, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 14.666666666666666},
 mean: 0.21678, std: 0.02962, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 16.0},
 mean: 0.15891, std: 0.05075, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 4.0},
 mean: 0.16957, std: 0.04363, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.17904, std: 0.04641, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.17634, std: 0.04313, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 8.0},
 mean: 0.18070, std: 0.04215, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.18228, std: 0.04064, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.18575, std: 0.04308, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 12.0},
 mean: 0.18909, std: 0.04002, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.18546, std: 0.04122, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.18941, std: 0.03956, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 16.0},
 mean: 0.11905, std: 0.06393, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 4.0},
 mean: 0.12714, std: 0.05933, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.12654, std: 0.06465, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.12593, std: 0.06096, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 8.0},
 mean: 0.13105, std: 0.05843, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.12967, std: 0.05950, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.13478, std: 0.05601, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 12.0},
 mean: 0.13623, std: 0.05727, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.13384, std: 0.05809, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.13661, std: 0.05929, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 16.0}]

gscv.best_score_

0.2898096239376933

gscv.best_params_

{'learning_rate': 0.01,
 'n_estimators': 1000,
 'scale_pos_weight': 9.333333333333332}
"""

"""
[mean: 0.28876, std: 0.02667, params: {'boosting_type': 'gbdt', 'learning_rate': 0.01, 'n_estimators': 750, 'scale_pos_weight': 9.0},
 mean:0.27868, std: 0.01830, params: {'boosting_type': 'dart', 'learning_rate': 0.01, 'n_estimators': 750, 'scale_pos_weight': 9.0},
 mean: 0.29242, std: 0.02722, params: {'boosting_type': 'goss', 'learning_rate': 0.01, 'n_estimators': 750, 'scale_pos_weight': 9.0}]

gscv.best_score_

0.29242072490189625 --> 0.33958

gscv.best_params_

{'boosting_type': 'goss',
 'learning_rate': 0.01,
 'n_estimators': 750,
 'scale_pos_weight': 9.0}
"""

In [22]:
# clf = CatBoostClassifier(iterations=9000)
#clf = DecisionTreeClassifier(random_state=17)
best_params = gscv.best_params_
best_params["random_state"] = 17
clf = lgb.LGBMClassifier(**best_params)
clf.fit(X, y)

LGBMClassifier(boosting_type='goss', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.01, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=750,
        n_jobs=-1, num_leaves=31, objective=None, random_state=17,
        reg_alpha=0.0, reg_lambda=0.0, scale_pos_weight=9.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=1)

In [38]:
clf = CatBoostClassifier(scale_pos_weight=8.255102040816327, random_seed=18)
clf.fit(X[:, fi > fi_sorted[200]], y)

CatboostError: catboost/libs/options/loss_description.h:81: f1 loss is not supported

In [7]:
%%time
clf = xgb.XGBClassifier(scale_pos_weight=8.255102040816327, n_estimators=300, gamma=10, random_state=17)
clf.fit(X, y)

CPU times: user 4min 46s, sys: 411 ms, total: 4min 46s
Wall time: 4min 47s


In [14]:
clf

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=10, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=17, reg_alpha=0, reg_lambda=1,
       scale_pos_weight=8.255102040816327, seed=None, silent=True,
       subsample=1)

In [15]:
prediction = clf.predict(test["x"]).astype(int)

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
np.unique(prediction, return_counts=True)

(array([0, 1]), array([3288,  757]))

In [17]:
df_sub = pd.read_csv("sample_submission.csv")
df_sub["mg"] = prediction
df_sub.to_csv("my_sub.csv", index=False)

In [13]:
df_sub

,Id,mg
0,0,1
1,1,1
2,2,1
3,3,0
4,4,0
5,5,0
6,6,1
7,7,0
8,8,1
9,9,1
